Imports



In [1]:
! pip install pymorphy2

In [2]:
import re
import numpy as np
import pandas as pd
from pymorphy2 import MorphAnalyzer

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\trekc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def clean(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('(https?://[^\s]+)', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = re.sub('rt', '', text)
    text = re.sub('[^а-яА-Я]+', ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text

In [6]:
d_pos = pd.read_csv('positive.csv', sep=';')
d_pos.columns = ['id', 'b', 'c', 'text', 'sent', 'f', 'g', 'h', 'i', 'j', 'k', 'l']
d_neg = pd.read_csv('negative.csv', sep=';')
d_neg.columns = ['id', 'b', 'c', 'text', 'sent', 'f', 'g', 'h', 'i', 'j', 'k', 'l']

In [7]:
dp = d_pos[['id','text']][:5000]
dn = d_neg[['id','text']][:5000]

In [8]:
new_df = pd.concat([dp, dn], ignore_index=True)
new_df 

,id,text
0,408906692693221377,"Да, все-таки он немного похож на него. Но мой ..."
1,408906695083954177,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
2,408906695356973056,"RT @digger2912: ""Кто то в углу сидит и погибае..."
3,408906761416867842,@irina_dyshkant Вот что значит страшилка :D\nН...
4,408906761769598976,ну любишь или нет? — Я не знаю кто ты бля:D ht...
...,...,...
9995,409766375364644864,ой(( я совсем забыла восстановить старый аккау...
9996,409766474010886144,Ну и где ночной твиттер? Все спят что ли? Так ...
9997,409766520009793536,"Девочка моя,спасибо тебе за этот вечер .мне оч..."
9998,409766520425054208,"@DianaPalicina долго объяснять:((( все плохо,вот"


In [9]:
new_df['text'] = [clean(t) for t in new_df['text']]
new_df

,id,text
0,408906692693221377,да все таки он немного похож на него но мой ма...
1,408906695083954177,ну ты идиотка я испугалась за тебя
2,408906695356973056,кто то в углу сидит и погибает от голода а мы ...
3,408906761416867842,вот что значит страшилка но блин посмотрев все...
4,408906761769598976,ну любишь или нет я не знаю кто ты бля
...,...,...
9995,409766375364644864,ой я совсем забыла восстановить старый аккаунт...
9996,409766474010886144,ну и где ночной твиттер все спят что ли так лю...
9997,409766520009793536,девочка моя спасибо тебе за этот вечер мне оче...
9998,409766520425054208,долго объяснять все плохо вот


In [10]:
def pymorphy_ana(text):
    lemmas = []
    pos = []
    text = word_tokenize(text)
    for word in text:
        if word != '.':
            ana = MorphAnalyzer().parse(word)
            lemmas.append(ana[0].normal_form)
            if ana[0].tag.POS != None:
                pos.append(ana[0].tag.POS) 
            else:
                pos.append('None')
    lemmas = ' '.join(lemmas)          
    pos = ' '.join(pos)
    return lemmas, pos

In [11]:
prep = [pymorphy_ana(i) for i in tqdm(new_df['text'])]

In [12]:
lemmas = [i[0] for i in prep]
pos = [i[1] for i in prep]

In [13]:
new_df['lemmas'] = lemmas
new_df['pos'] = pos

In [14]:
new_df

,id,text,lemmas,pos
0,408906692693221377,да все таки он немного похож на него но мой ма...,да всё таки он немного похожий на оно но мыть ...,PRCL PRCL PRCL NPRO ADVB ADJS PREP NPRO CONJ V...
1,408906695083954177,ну ты идиотка я испугалась за тебя,ну ты идиотка я испугаться за ты,PRCL NPRO NOUN NPRO VERB PREP NPRO
2,408906695356973056,кто то в углу сидит и погибает от голода а мы ...,кто то в угол сидеть и погибать от голод а мы ...,NPRO CONJ PREP NOUN VERB CONJ VERB PREP NOUN C...
3,408906761416867842,вот что значит страшилка но блин посмотрев все...,вот что значит страшилка но блин посмотреть вс...,PRCL CONJ CONJ NOUN CONJ INTJ GRND PRCL NOUN P...
4,408906761769598976,ну любишь или нет я не знаю кто ты бля,ну любить или нет я не знать кто ты бля,PRCL VERB CONJ PRED NPRO PRCL VERB NPRO NPRO INTJ
...,...,...,...,...
9995,409766375364644864,ой я совсем забыла восстановить старый аккаунт...,ой я совсем забыть восстановить старый аккаунт...,INTJ NPRO ADVB VERB INFN ADJF NOUN CONJ NOUN A...
9996,409766474010886144,ну и где ночной твиттер все спят что ли так лю...,ну и где ночной твиттер всё спать что ли так л...,PRCL CONJ ADVB ADJF NOUN PRCL VERB CONJ PRCL C...
9997,409766520009793536,девочка моя спасибо тебе за этот вечер мне оче...,девочка мой спасибо ты за этот вечер я очень т...,NOUN ADJF INTJ NPRO PREP ADJF NOUN NPRO ADVB N...
9998,409766520425054208,долго объяснять все плохо вот,долго объяснять всё плохо вот,ADVB INFN PRCL PRED PRCL


In [15]:
new_df.to_csv('dataset.csv', index=False)